In [229]:
from cgitb import text
from traceback import print_tb
from transformers import pipeline
from youtube_transcript_api import YouTubeTranscriptApi

print('Please Enter the link to the YouTube Video')
link = input()
video_id = link.split("=")[1]
transcript = YouTubeTranscriptApi.get_transcript(video_id)

full_transcript = ""
for x in transcript:
    full_transcript += " " + x['text']

print(len(full_transcript))

summerizar = pipeline("summarization")

itr = int(len(full_transcript)/400)
summary_text = []
for i in range(0, itr+1):
    start = 0
    start = i*400
    
    end = (i+1)*400
    out = summerizar(full_transcript[start:end], max_length=57)
    out = out[0]
    out = out["summary_text"]
    summary_text.append(out)

print(summary_text)

Please Enter the link to the YouTube Video
https://www.youtube.com/watch?v=rHvCQEr_ETk


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


476


Your max_length is set to 57, but you input_length is only 28. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)


[" The power of the dragon scroll is . . Mine! It's nothing! I didn't get it the first time either . There is no secret ingredient. It's just you . I'm gonna pee! Don't! Don’t! You’re just", " Luffing! Shifu didn't teach you that! Nope . I figured it out. Skadoosh! I figured out. luffing. luffsing! I'm not sure what I'm doing . I'm going to do it. I'm"]


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\hp\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
TypeError: iExit() missing 1 required positional argument: 'self'
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\hp\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
TypeError: iExit() missing 1 required positional argument: 'self'
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\hp\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
TypeError: iExit() missing 1 required positional argument: 'self'
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\hp\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
TypeError: iExit() missing 1 required positional argument: 'self'


In [249]:
from cgitb import text
from traceback import print_tb
from transformers import pipeline
from youtube_transcript_api import YouTubeTranscriptApi
from curses.textpad import Textbox
import string
from time import strftime, time
from tkinter import*
from tkinter import ttk
import tkinter
from PIL import Image,ImageTk,ImageDraw
from tkinter import messagebox
from datetime import datetime
from time import strftime
from pytz import timezone
root=tkinter.Tk()
root.grid_rowconfigure(0, weight=1)
root.grid_columnconfigure(0, weight=1)
root.geometry("1550x800+0+0")
root['bg']='black'
root.title("Youtube-Transcript Summarizer")

#========================================= time ==========================================================#
def current_time():
    now_utc = datetime.now(timezone('UTC'))
    now_asia = now_utc.astimezone(timezone('Asia/Kolkata'))
    string=now_asia.strftime('%H:%M:%S %p')
    lbl.config(text=string)
    lbl.after(1000,current_time)

lbl=Label(root,font=(SUNKEN,15,'bold'),fg='white',bg='black',border=0,borderwidth=0)
lbl.place(x=1,y=5)
current_time()
        
#======================================= Heading ========================================================#
img1=Image.open(r"image_resource\you_img.jpg")
img1=img1.resize((130,120),Image.ANTIALIAS)
img1=ImageTk.PhotoImage(img1)

hd_img1=Label(root,image=img1,bg='black')
hd_img1.place(x=1040,y=3.5)
hd_text=Label(root,text="Youtube Transcript Summarizer",font=(SUNKEN,30,'bold'),fg='white',bg='black',border=0,borderwidth=0)
hd_text.place(x=450,y=30)

#======================================== Exit Button ===================================================#
def exit_fun():
    root.destroy()
def iExit():
    iExit_t=messagebox.askyesno("Exit","Do you want to exit")
    if iExit_t>0:
        root.destroy()
    else:
        return    
exit_btn=Button(root,text='Quit',bg='red',command=iExit,font=('Arial',13,'bold'),fg='white',activebackground='red',activeforeground='white',border=0,borderwidth=3,cursor='hand2',relief=RIDGE)
exit_btn.place(x=1400,y=10,width=100,height=25)


#====================================== get-started & stop ==============================================#
def getlink():
    link=StringVar()
    def var_text():
        name=link.get()
        with open("link.txt", "w") as text_file:
            text_file.write(name)
        link.set("")

    link_lbl=Label(root,text="Enter URL",bg='black',fg='light blue',font=('SUNKEN',13,'bold'))
    link_lbl.place(x=100,y=250)
    link_data=ttk.Entry(root,width=20,textvariable=link,font=('SUNKEN',13,'normal'))
    link_data.place(x=200,y=250)
    sub_btn=tk.Button(root,text = 'Submit', command = var_text,bg='white',fg='black',border=1,borderwidth=1)
    sub_btn.place(x=400,y=250)
    
start_img=Image.open(r"image_resource\button_activate.png")
start_img=start_img.resize((204,55),Image.ANTIALIAS)
start_img=ImageTk.PhotoImage(start_img)
start=Button(root,command=getlink,image=start_img,bg='black',fg='red',cursor='hand2',border=0,borderwidth=0,activebackground='black')
start.place(x=670,y=200,width=207,height=57)

stop_img=Image.open(r"image_resource\button_stop (3).png")
stop_img=stop_img.resize((140,34),Image.ANTIALIAS)
stop_img=ImageTk.PhotoImage(stop_img)
stop=Button(root,image=stop_img,bg='black',fg='red',cursor='hand2',border=0,borderwidth=0,activebackground='black')
stop.place(x=710,y=700,width=150,height=37)
#========================================= text-area =====================================================#
txtarea = Text(root,bg='lavender',font=("TimesNewRoman",11),width=90, height=20,borderwidth=4,relief='solid',padx=10,pady=10,spacing2=5)
txtarea.pack(pady=100)
txtarea.place(x=400,y=300)
#======================================= Get Link & API call ======================================================#
myfile = open('link.txt', "r")
you_link = myfile.readline()
myfile.close()
def youtube():
    video_id = you_link.split("=")[1]
    transcript = YouTubeTranscriptApi.get_transcript(video_id)
    full_transcript = ""
    for x in transcript:
        full_transcript += " " + x['text']

    print(len(full_transcript))

    summerizar = pipeline("summarization")
    itr = int(len(full_transcript)/400)
    summary_text = []
    for i in range(0, itr+1):
        start = 0
        start = i*400
    
        end = (i+1)*400
        out = summerizar(full_transcript[start:end], max_length=57)
        out = out[0]
        out = out["summary_text"]
        summary_text.append(out)

    print(summary_text)
    
#======================================= Language-button ===============================================#
btn_eng=Image.open(r"image_resource\button_english.png")
btn_eng=btn_eng.resize((132,34),Image.ANTIALIAS)
btn_eng=ImageTk.PhotoImage(btn_eng)
eng=Button(root,image=btn_eng,bg='black',fg='red',cursor='hand2',border=0,borderwidth=0,activebackground='black')
eng.place(x=200,y=320,width=135,height=37)

btn_hindi=Image.open(r"image_resource\button_hindi.png")
btn_hindi=btn_hindi.resize((132,34),Image.ANTIALIAS)
btn_hindi=ImageTk.PhotoImage(btn_hindi)
hindi=Button(root,image=btn_hindi,bg='black',fg='red',cursor='hand2',border=0,borderwidth=0,activebackground='black')
hindi.place(x=200,y=420,width=135,height=37)

btn_german=Image.open(r"image_resource\button_german.png")
btn_german=btn_german.resize((132,34),Image.ANTIALIAS)
btn_german=ImageTk.PhotoImage(btn_german)
german=Button(root,image=btn_german,bg='black',fg='red',cursor='hand2',border=0,borderwidth=0,activebackground='black')
german.place(x=200,y=520,width=135,height=37)

btn_french=Image.open(r"image_resource\button_french.png")
btn_french=btn_french.resize((132,34),Image.ANTIALIAS)
btn_french=ImageTk.PhotoImage(btn_french)
french=Button(root,image=btn_french,bg='black',fg='red',cursor='hand2',border=0,borderwidth=0,activebackground='black')
french.place(x=200,y=620,width=135,height=37)

btn_bengali=Image.open(r"image_resource\button_bengali.png")
btn_bengali=btn_bengali.resize((132,34),Image.ANTIALIAS)
btn_bengali=ImageTk.PhotoImage(btn_bengali)
bengali=Button(root,image=btn_bengali,bg='black',fg='red',cursor='hand2',border=0,borderwidth=0,activebackground='black')
bengali.place(x=1210,y=320,width=135,height=37)

btn_punjabi=Image.open(r"image_resource\button_punjabi.png")
btn_punjabi=btn_punjabi.resize((132,34),Image.ANTIALIAS)
btn_punjabi=ImageTk.PhotoImage(btn_punjabi)
punjabi=Button(root,image=btn_punjabi,bg='black',fg='red',cursor='hand2',border=0,borderwidth=0,activebackground='black')
punjabi.place(x=1210,y=420,width=135,height=37)

btn_tamil=Image.open(r"image_resource\button_tamil.png")
btn_tamil=btn_tamil.resize((132,34),Image.ANTIALIAS)
btn_tamil=ImageTk.PhotoImage(btn_tamil)
tamil=Button(root,image=btn_tamil,bg='black',fg='red',cursor='hand2',border=0,borderwidth=0,activebackground='black')
tamil.place(x=1210,y=520,width=135,height=37)

btn_telugu=Image.open(r"image_resource\button_telugu.png")
btn_telugu=btn_telugu.resize((132,34),Image.ANTIALIAS)
btn_telugu=ImageTk.PhotoImage(btn_telugu)
telugu=Button(root,image=btn_telugu,bg='black',fg='red',cursor='hand2',border=0,borderwidth=0,activebackground='black')
telugu.place(x=1210,y=620,width=135,height=37)


root.mainloop()  

In [260]:
!git remote add origin https://github.com/ups1610/Text-summarizer.git

error: remote origin already exists.


In [265]:
!git push --set-upstream origin main

error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/ups1610/Text-summarizer.git'


In [253]:
from cgitb import text
from traceback import print_tb
from transformers import pipeline
from youtube_transcript_api import YouTubeTranscriptApi
myfile = open('link.txt', "r")
you_link = myfile.readline()
myfile.close()
def youtube():
    video_id = you_link.split("=")[1]
    transcript = YouTubeTranscriptApi.get_transcript(video_id)
    full_transcript = ""
    for x in transcript:
        full_transcript += " " + x['text']

    print(len(full_transcript))

    summerizar = pipeline("summarization")
    itr = int(len(full_transcript)/400)
    summary_text = []
    for i in range(0, itr+1):
        start = 0
        start = i*400
    
        end = (i+1)*400
        out = summerizar(full_transcript[start:end], max_length=57)
        out = out[0]
        out = out["summary_text"]
        summary_text.append(out)

    print(summary_text)
youtube()

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


476


Your max_length is set to 57, but you input_length is only 28. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)


[" The power of the dragon scroll is . . Mine! It's nothing! I didn't get it the first time either . There is no secret ingredient. It's just you . I'm gonna pee! Don't! Don’t! You’re just", " Luffing! Shifu didn't teach you that! Nope . I figured it out. Skadoosh! I figured out. luffing. luffsing! I'm not sure what I'm doing . I'm going to do it. I'm"]


In [241]:
var

NameError: name 'var' is not defined